In [1]:
import json
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from tqdm import tqdm
from scipy.spatial import distance_matrix

In [2]:
with open('data/contest_input.json') as f:
#with open('data/hard_input.json') as f:
    contest_data = json.load(f)

In [3]:
contest_data.keys()

dict_keys(['depots', 'orders', 'couriers'])

In [4]:
couriers = pd.DataFrame(contest_data['couriers'])
orders = pd.DataFrame(contest_data['orders'])
depots = pd.DataFrame(contest_data['depots'])

In [5]:
def filter_orders(orders):
    orders = orders[(orders['pickup_to']>=360) 
                    & (orders['dropoff_to'] >=360) 
                    & (orders['payment'] > 0) 
                    & (orders['dropoff_to'] >= orders['pickup_to'])]
    return orders

In [6]:
orders = filter_orders(orders)

In [7]:
couriers_x = couriers.location_x
couriers_y = couriers.location_y

In [8]:
def manh_distance(x0,y0,x1,y1):
    return np.abs(x0 - x1) + np.abs(y0-y1)

def courier_distance(x0,y0,x1,y1):
    return 10 + manh_distance(x0,y0,x1,y1)

In [65]:
%%time
drop_x = orders['dropoff_location_x'].values
drop_y = orders['dropoff_location_y'].values
pick_x = orders['pickup_location_x'].values
pick_y = orders['pickup_location_y'].values

money = orders['payment'].values
end2start = np.zeros((orders.shape[0], orders.shape[0]))
end2start_time = np.zeros((orders.shape[0], orders.shape[0]))
drop_coordinates = orders[['dropoff_location_x', 'dropoff_location_y']].values
pick_coordinates = orders[['pickup_location_x', 'pickup_location_y']].values
time_from_start_to_end = np.array(courier_distance(pick_x, pick_y, drop_x, drop_y))
# [end_id, start_id]
time_from_ends_to_starts = 10 + distance_matrix(drop_coordinates, pick_coordinates, p=1).astype('float')
# Avoid to go to myself
time_from_ends_to_starts += np.eye(time_from_ends_to_starts.shape[0]) * 10e6
# end0 -> start1 -> end1

time_from_ends_to_end = time_from_start_to_end + time_from_ends_to_starts
#
metric_from_ends_to_end = np.array(money) - time_from_ends_to_end * 2

drop_from = orders['dropoff_from'].values
drop_to = orders['dropoff_to'].values
pick_from = orders['pickup_from'].values
pick_to = orders['pickup_to'].values

CPU times: user 1.72 s, sys: 985 ms, total: 2.71 s
Wall time: 2.71 s


In [66]:
def greedy_courier(start_position, visited_nodes, PROFIT_THRESHOLD=50):
    """
    Need a lot of global variables
    """
    path = []
    pick_time_first = 10 + distance_matrix([start_position], pick_coordinates, p=1).reshape((-1,))
    drop_time_first = pick_time_first + time_from_start_to_end
    metric_first = money - (pick_time_first + drop_time_first) * 2
    current_time = 360
    sorted_metric = np.argsort(metric_first)[::-1]
    # first step
    for j, id_ in enumerate(sorted_metric):
        if id_ in visited_nodes:
            continue
        pick_time = current_time + pick_time_first[id_]
        drop_time = current_time + drop_time_first[id_]
        
        if metric_first[id_] < -10:
            break
        if  pick_from[id_] < pick_time < pick_to[id_] and drop_from[id_] < drop_time < drop_to[id_]:
            path.append(id_)
            current_time = drop_time
            metric_from_ends_to_end[:, id_] = -10e5
            break
        elif pick_time < pick_from[id_]: # Let's wait
            wait_pick = pick_from[id_] - pick_time
            pick_time = pick_from[id_]
            new_drop_time = pick_time + time_from_start_to_end[id_]
            if new_drop_time < drop_to[id_]: # Can drop
                if drop_from[id_] <= new_drop_time: # In time!
                    if (metric_first[id_] - wait_pick * 2) > -50:
                        path.append(id_)
                        current_time = new_drop_time
                        metric_from_ends_to_end[:, id_] = -10e5
                        break
                else: # Should wait drop
                    wait_drop = drop_from[id_] - new_drop_time
                    if (metric_first[id_] - wait_pick * 2 - wait_drop * 2) > -50:
                        path.append(id_)
                        current_time = new_drop_time + wait_drop
                        metric_from_ends_to_end[:, id_] = -10e5
                        break
    STOP = False
    if len(path) == 0:
        STOP = True
    else:
        visited_nodes.add(path[0])
    while not STOP:
        current_metric_array = metric_from_ends_to_end[path[-1], :]
        sorted_metric = np.argsort(current_metric_array)[::-1]
        for j, id_ in enumerate(sorted_metric):
            pick_time = current_time + time_from_ends_to_starts[path[-1]][id_]
            drop_time = pick_time + time_from_start_to_end[id_]
            if  current_metric_array[id_] < -10 or current_time >=1439:
                STOP = True
            if  pick_from[id_] < pick_time < pick_to[id_] and drop_from[id_] < drop_time < drop_to[id_] and id_ not in visited_nodes:
                path.append(id_)
                visited_nodes.add(id_)
                current_time = drop_time
                metric_from_ends_to_end[:, id_] = -10e5
                break  
            elif pick_time < pick_from[id_]: # Let's wait
                wait_pick = pick_from[id_] - pick_time
                pick_time = pick_from[id_]
                new_drop_time = pick_time + time_from_start_to_end[id_]
                if new_drop_time < drop_to[id_]: # Can drop
                    if drop_from[id_] <= new_drop_time: # In time!
                        if (current_metric_array[id_] - wait_pick * 2) > PROFIT_THRESHOLD:
                            path.append(id_)
                            visited_nodes.add(id_)
                            current_time = new_drop_time
                            metric_from_ends_to_end[:, id_] = -10e5
                            break
                    else: # Should wait drop
                        wait_drop = drop_from[id_] - new_drop_time
                        if (current_metric_array[id_] - wait_pick * 2 - wait_drop * 2) > PROFIT_THRESHOLD:
                            path.append(id_)
                            visited_nodes.add(id_)
                            current_time = new_drop_time + wait_drop
                            metric_from_ends_to_end[:, id_] = -10e5
                            break
    visited_nodes = visited_nodes
    return path, current_time, visited_nodes

In [67]:
paths, times = [], []
visited_nodes = set()
for start_position in tqdm(couriers[['location_x', 'location_y']].values):
    path, time, visited_nodes = greedy_courier(start_position, visited_nodes=visited_nodes)
    paths.append(path)
    times.append(time)

100%|██████████| 300/300 [00:11<00:00, 35.07it/s]


In [68]:
order_ids = orders['order_id'].values
pickup_point_ids = orders['pickup_point_id'].values
dropoff_point_ids = orders['dropoff_point_id'].values

In [69]:
orders[:2]

,dropoff_from,dropoff_location_x,dropoff_location_y,dropoff_point_id,dropoff_to,order_id,payment,pickup_from,pickup_location_x,pickup_location_y,pickup_point_id,pickup_to
0,600,252,197,60001,960,10001,313,480,284,235,40001,570
1,630,24,105,60002,660,10002,519,420,244,262,40002,450


In [70]:
actions = []
for path, courier_id in zip(paths, np.arange(1,301)):
    for id_ in path:
        actions.append([courier_id, "pickup", order_ids[id_], pickup_point_ids[id_]])
        actions.append([courier_id, "dropoff", order_ids[id_], dropoff_point_ids[id_]])
        
        #actions.append({ "courier_id": int(courier_id),
        #                "action": "pickup",
        #                  "order_id": int(order_ids[id_]),
        #                  "point_id": int(pickup_point_ids[id_])})
        #actions.append({ "courier_id": int(courier_id),
        #                "action": 'dropoff',
        #                  "order_id": int(order_ids[id_]),
        #                  "point_id": int(dropoff_point_ids[id_])})

In [71]:
result = pd.DataFrame(actions, columns=['courier_id', 'action', 'order_id', 'point_id'])

In [72]:
result.to_json('result_contest.json', orient='records')

In [73]:
not_started = 0
for path in paths:
    if len(path) == 0:
        not_started +=1 

In [74]:
not_started

46

In [75]:
def get_profit(path,time):
    total = 0
    for id_ in path:
        total += money[id_]
    return total - (time - 360) * 2
profits = []
for path, time in zip(paths, times):
    profits.append(get_profit(path,time))
profits = np.array(profits)
np.sum(profits)

216599.0

164942.0

In [134]:
visited_nodes = visited_nodes | set(paths[0])

In [135]:
visited_nodes

{1289, 1791, 1810, 3896, 6524, 6760, 6814, 6973, 7001, 7143}

In [29]:
def greedy_courier(start_position, visited_nodes):
    """
    Need a lot of global variables
    """
    path = []
    pick_time_first = 10 + distance_matrix([start_position], pick_coordinates, p=1).reshape((-1,))
    drop_time_first = pick_time_first + time_from_start_to_end
    metric_first = money - (pick_time_first + drop_time_first) * 2
    current_time = 360
    sorted_metric = np.argsort(metric_first)[::-1]
    # first step
    for j, id_ in enumerate(sorted_metric):
        if id_ in visited_nodes:
            continue
        pick_time = current_time + pick_time_first[id_]
        drop_time = current_time + drop_time_first[id_]
        if metric_first[id_] < -10:
            break
        if  pick_from[id_] < pick_time < pick_to[id_] and drop_from[id_] < drop_time < drop_to[id_]:
            path.append(id_)
            current_time = drop_time
            metric_from_ends_to_end[:, id_] = -10e5
            break
    STOP = False
    if len(path) == 0:
        STOP = True
    else:
        visited_nodes.add(path[0])
    while not STOP:
        current_metric_array = metric_from_ends_to_end[path[-1], :]
        sorted_metric = np.argsort(current_metric_array)[::-1]
        for j, id_ in enumerate(sorted_metric):
            pick_time = current_time + time_from_ends_to_starts[path[-1]][id_]
            drop_time = pick_time + time_from_start_to_end[id_]
            if  current_metric_array[id_] < -10 or current_time >=1439:
                STOP = True
            if  pick_from[id_] < pick_time < pick_to[id_] and drop_from[id_] < drop_time < drop_to[id_] and id_ not in visited_nodes:
                path.append(id_)
                visited_nodes.add(id_)
                current_time = drop_time
                metric_from_ends_to_end[:, id_] = -10e5
                break  
    visited_nodes = visited_nodes
    return path, current_time, visited_nodes